In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
path0 = r'C:\Users\siyoung\Desktop\Korea Data Project'
path1 = r'C:\Users\siyoung\Desktop\Korea Data Project\code_siyoung'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
kcb_var = pd.read_excel(os.path.join(path0,'Copy of KCB GranData Variable List (as of Feb. 8 2024)-2.xlsx'), sheet_name='KCB')['컬럼명'][17:].drop(columns=['HSHD_SEQNO']).to_list()
shc_var = pd.read_excel(os.path.join(path0,'Copy of KCB GranData Variable List (as of Feb. 8 2024)-2.xlsx'), sheet_name='Shinhan Card')['컬럼명'][4:65].to_list()
skt_var = pd.read_excel(os.path.join(path0,'Copy of KCB GranData Variable List (as of Feb. 8 2024)-2.xlsx'), sheet_name='SKTelecom')['컬럼명'].dropna()[3:].to_list()

In [5]:
ds_random = pd.read_parquet(os.path.join(path0,'big_data.parquet'))

ds_birth = pd.read_parquet(os.path.join(path0,'big_data.parquet'))
ds_birth['birth_flag'] = ds_birth.groupby('HSHD_SEQNO')['shc_ch_birth'].transform('max')
ds_birth = ds_birth[ds_birth['birth_flag'] == 1]

In [63]:
identifying_var = ['KEY','AGE','SEX','BS_YR_MON']
ds_random_kcb = ds_random[identifying_var + ['HOME_ADM']+[col for col in ds_random.columns if col in kcb_var]]
ds_random_shc = ds_random[identifying_var+['ta_ym','HSHD_SEQNO'] + [col for col in ds_random.columns if col in shc_var]]
ds_random_skt = ds_random[identifying_var+ ['GENDER','HSHD_SEQNO']+ [col for col in ds_random.columns if col in skt_var]]

ds_birth_kcb = ds_birth[identifying_var + ['HOME_ADM'] + [col for col in ds_birth.columns if col in kcb_var]]
ds_birth_shc = ds_birth[identifying_var +['ta_ym','HSHD_SEQNO'] + [col for col in ds_birth.columns if col in shc_var]]
ds_birth_skt = ds_birth[identifying_var + ['GENDER','HSHD_SEQNO'] + [col for col in ds_birth.columns if col in skt_var]]

In [64]:
def skt_preprocess(skt_data):
    
    '''
    SKT Data marks N/A for 0 values
    This Function fills 0 to N/A values if One consented to marketing in 12/2023
    '''


    skt_data['DAY_USAGE_FLAG'] = skt_data['GENDER'].notna().astype('int')
    

    # At Now, we fill variables ends with '_DAY'
    day_cols = [col for col in skt_data.columns if col.endswith('_DAY')]
    mask = (skt_data['DAY_USAGE_FLAG'] == 1)
    skt_data.loc[mask,day_cols] = skt_data.loc[mask, day_cols].fillna(0)

    score_cols = [col for col in skt_data.columns if col.endswith('_SCORE')]
    skt_data.loc[mask,score_cols] = skt_data.loc[mask, score_cols].fillna(0)

    consent_mask = skt_data['DAY_USAGE_FLAG'].notna()

    try:
        skt_data.loc[consent_mask, 'BF_OVERDUE_CNT'] = skt_data.loc[consent_mask, 'BF_OVERDUE_CNT'].fillna(0)
    
    except KeyError:
       pass


    return skt_data


def clean_data(data):

    data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')
    return data

# Data Process

In [65]:
ds_random_shc = clean_data(ds_random_shc)
shc_hshd_list = ds_random_shc[ds_random_shc['ta_ym'].notna()].HSHD_SEQNO.unique()
ds_random_shc = ds_random_shc[ds_random_shc['HSHD_SEQNO'].isin(shc_hshd_list)]
ds_random_shc.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace=True)

C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')


In [66]:
ds_random_shc['is_f_20_40'] = (ds_random_shc['SEX'] == 'Female') & (ds_random_shc['AGE'].between(20,40))
ds_random_shc['is_m_20_40'] = (ds_random_shc['SEX'] == 'Male') & (ds_random_shc['AGE'].between(20,40))

ds_random_shc['hshd_is_two_parents'] = ((ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_shc['n_obs'] = ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_shc = ds_random_shc[(ds_random_shc['hshd_is_two_parents'] == True) &
                              (ds_random_shc['n_obs'] == 2)]
ds_random_shc.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)


# birth 제외하기
ds_random_shc['hshd_has_no_birth'] = (ds_random_shc.groupby('HSHD_SEQNO')['shc_ch_birth'].transform('max') == 0)
ds_random_shc = ds_random_shc[ds_random_shc['hshd_has_no_birth']]

In [67]:
ds_random_shc.shape

(432, 72)

In [68]:
ds_random_kcb = clean_data(ds_random_kcb)
ds_random_kcb = ds_random_kcb[ds_random_kcb.HSHD_SEQNO.isin(shc_hshd_list)] #row 수 같아야 함
ds_random_kcb.shape

C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')


(447768, 132)

In [69]:
ds_random_kcb['is_f_20_40'] = (ds_random_kcb['SEX'] == 'Female') & (ds_random_kcb['AGE'].between(20,40))
ds_random_kcb['is_m_20_40'] = (ds_random_kcb['SEX'] == 'Male') & (ds_random_kcb['AGE'].between(20,40))

ds_random_kcb['hshd_is_two_parents'] = ((ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_kcb['n_obs'] = ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_kcb = ds_random_kcb[(ds_random_kcb['hshd_is_two_parents'] == True) &
                              (ds_random_kcb['n_obs'] == 2)]
ds_random_kcb.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)

In [70]:
ds_random_kcb

,KEY,AGE,SEX,BS_YR_MON,HOME_ADM,HSHD_SEQNO,ICM,ICM_RT,TOT_ASST,NET_ASST,RCNT_CAR_LN_CONT,FST_CAR_YM,RCNT_CAR_YM,HSHD_CNT,HOME_CHAR_DIST,COM_CHAR_DIST,OWN_HOUS_CNT,FAM_OWN_HOUS_CNT,LN_BAL,HOUS_LN_BAL,CRDT_LN_BAL,CAR_LN_BAL,LN_CONT_AMT,HOUS_LN_CONT_AMT,CRDT_LN_CONT_AMT,CAR_LN_CONT_AMT,LN_CNT,HOUS_LN_CNT,CRDT_LN_CNT,CAR_LN_CNT,LN_CONT_AMT_NEW,HOUS_LN_CONT_AMT_NEW,CRDT_LN_CONT_AMT_NEW,CAR_LN_CONT_AMT_NEW,LN_CNT_NEW,HOUS_LN_CNT_NEW,CRDT_LN_CNT_NEW,CAR_LN_CNT_NEW,CD_USE_AMT,SIN_CD_USE_AMT,CHK_CD_USE_AMT,CD_FUL_USE_AMT,CD_INSTL_USE_AMT,CD_CA_USE_AMT,CD_ABRD_USE_AMT,SIN_CD_CNT,CHK_CD_CNT,FST_HOUS_BUY_YM,BUY_HOUS_ADM,BUY_HOUS_PRICE,SHP_CNT,MX_SHP_ADM,SHP_TP1,SHP_TP2,SHP_CD_USE,SHP_LN_BAL,SHP_LN_CONT_AMT,SHP_LN_CNT,YR_SALES,DLQ_CNT,DLQ_AMT,DLQ_DAYS,DLQ_CNT_BNK,DLQ_CNT_CD,DLQ_CNT_CAP,DLQ_CNT_INS,DLQ_CNT_SAV,DLQ_AMT_BNK,DLQ_AMT_CD,DLQ_AMT_CAP,DLQ_AMT_INS,DLQ_AMT_SAV,DLQ_DAYS_BNK,DLQ_DAYS_CD,DLQ_DAYS_CAP,DLQ_DAYS_INS,DLQ_DAYS_SAV,LN_DLQ_CNT,LN_DLQ_AMT,LN_DLQ_DAYS,CDU_DLQ_CNT,CDU_DLQ_AMT,CDU_DLQ_DAYS,RCNT_HOME_CHG_YM,LN_BAL_BNK,LN_BAL_CD,LN_BAL_CAP,LN_BAL_INS,LN_BAL_SAV,LN_CONT_AMT_BNK,LN_CONT_AMT_CD,LN_CONT_AMT_CAP,LN_CONT_AMT_INS,LN_CONT_AMT_SAV,LN_CNT_BNK,LN_CNT_CD,LN_CNT_CAP,LN_CNT_INS,LN_CNT_SAV,LN_CONT_AMT_BNK_NEW,LN_CONT_AMT_CD_NEW,LN_CONT_AMT_CAP_NEW,LN_CONT_AMT_INS_NEW,LN_CONT_AMT_SAV_NEW,LN_CNT_BNK_NEW,LN_CNT_CD_NEW,LN_CNT_CAP_NEW,LN_CNT_INS_NEW,LN_CNT_SAV_NEW,CRDT_LN_REPAY,HOUS_LN_REPAY,HOUS_MRTY_MON,LTV,HSHD_LTV,DTI,K_SCORE,LN_DAY_CNT,LN_DAY_CNT_BNK,LN_DAY_CNT_CD,LN_DAY_CNT_CAP,LN_DAY_CNT_INS,LN_DAY_CNT_SAV,HOUS_LN_DAY_CNT,CRDT_LN_DAY_CNT,CAR_LN_DAY_CNT,CAR_OWN,CAR_SIZE,RCNT_CAR_BRND,CAR_OWN_TP,ELE_CAR_YN,OWN_LIV_YN,FAM_OWN_LIV_YN,is_f_20_40,is_m_20_40,hshd_is_two_parents,n_obs
186840,#lbx2x75iqx,25,Male,2019-01-01,44133310,#b3fgs76p,8301,37209,26827,29402,75115,29811,59363,15508,5543,28290,77223,17462,76283,32619,6116,27262,68565,76400,2782,15032,62798,70900,86157,46501,24610,90376,77604,40127,48690,74669,31563,94897,6185,69585,42043,30099,95484,24018,31072,6389,48456,60967,17022,84898,61650,45403,163,24742,1488,96115,63236,23470,56724,52765,74707,63542,81844,12459,90987,79459,55572,83094,6951,47193,22375,38814,16541,66481,97201,38530,76485,45069,99913,24534,31316,49589,37749,11094,87791,46675,85049,78130,47150,12306,51590,37318,30841,83828,32580,95960,88875,74839,27413,46327,35588,45259,13241,18245,89071,37418,65907,91091,2977,34654,14817,71530,61641,50621,25524,60149,98880,823,69690,64897,33098,24833,98591,1396,97792,1,2,1,0,1,0,2,False,True,True,2
186912,#vpddb3pxrm,35,Female,2019-01-01,44133310,#b3fgs76p,13464,97985,14864,88622,76104,78203,4345,32043,95687,69557,57952,26875,78918,78613,50301,28824,55617,94331,62557,99052,43083,78916,17687,33879,95334,41752,40665,87385,8226,69178,54490,40164,26763,91658,97634,61920,7720,7624,86250,96683,79861,4069,17766,50442,62515,9825,37593,96192,83695,59912,26411,64144,45592,3842,8743,96805,3921,62417,22394,39443,46800,28891,52885,68713,31830,79261,63272,94794,90951,79759,15076,12067,80055,61844,28228,51912,59764,33656,71150,60117,76538,77374,80910,43640,42494,13265,95189,17990,81361,11808,77063,83326,55487,41417,37647,94930,36318,49762,46469,72696,18934,30676,73111,63314,47396,33558,53374,89889,53062,33710,33122,38559,23171,34476,19871,74536,71031,80301,49800,1,2,0,2,0,1,1,True,False,True,2
186841,#lbx2x75iqx,25,Male,2019-02-01,44133310,#b3fgs76p,37440,74502,40817,11872,20208,76548,35066,33319,83221,84473,50650,76597,9993,87594,72743,39794,91797,99575,39732,18109,3140,13677,511,94843,11673,65845,27833,31330,21743,9983,30659,43657,75307,48225,2519,60925,99594,8569,36969,84949,42065,27709,22214,15688,51877,19968,45714,65059,22076,66987,53109,44672,54736,36172,94519,41381,88944,99189,21365,27575,69882,18814,72853,43587,39898,82916,42483,4237,95098,86391,77535,88224,33627,65294,6996,17673,778,2974,90369,46942,21706,17551,40909,71744,37394,93985,20569,28908,91267,11789,76424,54892,33242,11879,685,65365,66052,40021,29299,49376,53380,56370,30131,160,8422,23528,82767,90172,95899,71181,18108,7347

In [71]:
ds_random_hshd = ds_random_kcb.groupby(['HSHD_SEQNO', 'BS_YR_MON']).agg({'ICM':'sum', 'TOT_ASST':'sum'}).reset_index()
ds_random_hshd = ds_random_hshd.groupby('HSHD_SEQNO').agg({'ICM':'mean', 'TOT_ASST':'mean'}).reset_index()
_, income_bins = pd.qcut(ds_random_hshd['ICM'], q=5, retbins=True, duplicates='drop')
_, asset_bins = pd.qcut(ds_random_hshd['TOT_ASST'], q=5, retbins=True, duplicates='drop')

kcb_cutoffs = pd.DataFrame({
    **{f'ICM_cutoff{i}': [v] for i, v in enumerate(income_bins)},
    **{f'TOT_ASST_cutoff{i}': [v] for i, v in enumerate(asset_bins)}
})

#실제에서는 저장하기
# kcb_cutoffs.to_excel()

In [96]:
ds_random_hshd

,HSHD_SEQNO,ICM,TOT_ASST
0,#b3fgs76p,105934.791667,108942.500000
1,#bk0gx859,103208.416667,102848.569444
2,#bqk3yl8m,107408.861111,101425.444444
3,#bs8jgato,106111.111111,107086.319444
4,#bzw8fvf3,102776.875000,95642.375000
5,#c3u4ocp1,100046.444444,99299.250000
6,#d0ifmnb6,89504.736111,95893.569444
7,#d9zzbkj1,95331.958333,97865.958333
8,#dgp4j8y6,100938.680556,101404.333333
9,#dujt5u9j,107373.986111,100813.069444


In [72]:
ds_random_kcb.shape

(5328, 136)

In [73]:
ds_random_combined = ds_random_kcb.merge(ds_random_shc, on=['KEY','HSHD_SEQNO','BS_YR_MON'], how='inner')

In [74]:
home_mode = (
    ds_random_combined
    .groupby("HSHD_SEQNO")["HOME_ADM"]
    .agg(lambda x: x.mode().iloc[0])
    .reset_index()
)
home_mode['HOME_ADM'] = home_mode["HOME_ADM"].astype(str)
home_mode['HOME_ADM_2'] = home_mode['HOME_ADM'].str[:2]
home_mode['HOME_ADM_5'] = home_mode['HOME_ADM'].str[:5]
home_mode.rename(columns={'HOME_ADM':'HOME_ADM_b'}, inplace=True)

ds_random_combined = ds_random_combined.merge(home_mode, on=['HSHD_SEQNO'], how='left')

In [75]:
ds_random_skt = clean_data(ds_random_skt)
ds_random_skt = skt_preprocess(ds_random_skt)
ds_random_skt = ds_random_skt[ds_random_skt.HSHD_SEQNO.isin(shc_hshd_list)]

C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')
C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skt_data['DAY_USAGE_FLAG'] = skt_data['GENDER'].notna().astype('int')


In [76]:
ds_random_skt.shape

(447768, 184)

In [77]:
ds_random_skt['is_f_20_40'] = (ds_random_skt['SEX'] == 'Female') & (ds_random_skt['AGE'].between(20,40))
ds_random_skt['is_m_20_40'] = (ds_random_skt['SEX'] == 'Male') & (ds_random_skt['AGE'].between(20,40))

ds_random_skt['hshd_is_two_parents'] = ((ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_skt['n_obs'] = ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_skt = ds_random_skt[(ds_random_skt['hshd_is_two_parents'] == True) &
                              (ds_random_skt['n_obs'] == 2)]
ds_random_skt.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)

In [78]:
ds_random_indiv = ds_random_skt.groupby(['KEY']).agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'}).reset_index()
ds_random_combined = ds_random_combined.merge(ds_random_indiv, on = 'KEY', how='left')

In [79]:
# Cutoff value 구하기

_, sns_bins = pd.qcut(ds_random_indiv['COMM_SNS_DAY'], q=2, retbins=True, duplicates='drop')
_, portal_bins = pd.qcut(ds_random_indiv['INFO_PORTAL_DAY'], q=2, retbins=True, duplicates='drop')

skt_cutoffs = pd.DataFrame({
    **{f'sns_cutoff{i}': [v] for i, v in enumerate(sns_bins)},
    **{f'portal_cutoff{i}': [v] for i, v in enumerate(portal_bins)}
})
# skt_cutoffs.to_excel()

In [80]:
# Birth Household
ds_birth_shc = clean_data(ds_birth_shc)
birth_hshd_list = ds_birth_shc[ds_birth_shc['ta_ym'].notna()].HSHD_SEQNO.unique()
print(ds_birth_shc.shape)
ds_birth_shc = ds_birth_shc[ds_birth_shc.HSHD_SEQNO.isin(birth_hshd_list)]
print(ds_birth_shc.shape)

C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')


(717408, 67)
(445176, 67)


In [81]:
df_hshd_birth_months = ds_birth_shc[ds_birth_shc['shc_ch_birth'] == 1][['HSHD_SEQNO','BS_YR_MON']].drop_duplicates()
df_hshd_birth_months.sort_values(['HSHD_SEQNO','BS_YR_MON'], inplace=True)

df_hshd_birth_months['diff_months'] = df_hshd_birth_months.groupby(['HSHD_SEQNO'])['BS_YR_MON'].transform(lambda x: (x.dt.year - x.shift().dt.year)*12 + (x.dt.month - x.shift().dt.month))
df_hshd_birth_months = df_hshd_birth_months[(df_hshd_birth_months['diff_months'].isna())| (df_hshd_birth_months['diff_months'] > 10)]

df_hshd_birth_months.drop(columns={'diff_months'}, inplace = True)

df_hshd_birth_months['birth_order'] = df_hshd_birth_months.groupby('HSHD_SEQNO')\
    .cumcount() + 1

df_hshd_birth_months = df_hshd_birth_months[df_hshd_birth_months['birth_order'] == 1].drop(columns=['birth_order'])
df_hshd_birth_months.rename(columns={'BS_YR_MON':'BS_YR_MON_birth'}, inplace=True)
ds_birth_shc = ds_birth_shc.merge(df_hshd_birth_months, on=['HSHD_SEQNO'], how='left')


In [82]:
ds_birth_shc['is_f_20_40'] = (ds_birth_shc['SEX'] == 'Female') & (ds_birth_shc['AGE'].between(20,40))
ds_birth_shc['is_m_20_40'] = (ds_birth_shc['SEX'] == 'Male') & (ds_birth_shc['AGE'].between(20,40))

ds_birth_shc['hshd_is_two_parents'] = ((ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_birth_shc['n_obs'] = ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_birth_shc = ds_birth_shc[(ds_birth_shc['hshd_is_two_parents'] == True) &
                              (ds_birth_shc['n_obs'] == 2)]
ds_birth_shc.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)

In [83]:
ds_birth_shc.shape

(4896, 72)

In [84]:
ds_birth_kcb = clean_data(ds_birth_kcb)
ds_birth_kcb = ds_birth_kcb.loc[ds_birth_kcb.HSHD_SEQNO.isin(birth_hshd_list)]
print(ds_birth_kcb.shape)

(445176, 132)


C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\1588797076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BS_YR_MON'] = pd.to_datetime(data['BS_YR_MON'], format='%Y-%m')


In [85]:
ds_birth_kcb['is_f_20_40'] = (ds_birth_kcb['SEX'] == 'Female') & (ds_birth_kcb['AGE'].between(20,40))
ds_birth_kcb['is_m_20_40'] = (ds_birth_kcb['SEX'] == 'Male') & (ds_birth_kcb['AGE'].between(20,40))

ds_birth_kcb['hshd_is_two_parents'] = ((ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_birth_kcb['n_obs'] = ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_birth_kcb = ds_birth_kcb[(ds_birth_kcb['hshd_is_two_parents'] == True) &
                              (ds_birth_kcb['n_obs'] == 2)]
ds_birth_kcb.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)

In [86]:
ds_birth_kcb.shape #row shc와 비교

(4896, 136)

In [87]:
ds_birth_combined = ds_birth_kcb.merge(ds_birth_shc, on=['KEY','HSHD_SEQNO','BS_YR_MON']
                                       ,how='inner')

In [ ]:
ds_birth_combined['months_from_birth'] = ((ds_birth_combined['BS_YR_MON'].dt.year - ds_birth_combined['BS_YR_MON_birth'].dt.year) * 12 +
                                   (ds_birth_combined['BS_YR_MON'].dt.month - ds_birth_combined['BS_YR_MON_birth'].dt.month))

ds_birth_combined['max_months_from_birth_before'] = (
    ds_birth_combined
    .assign(tmp=lambda d: d['months_from_birth'].where(d['months_from_birth'] < 0))
    .groupby('HSHD_SEQNO')['tmp']
    .transform('max')
)

hshd_birth_adm = ds_birth_combined[ds_birth_combined['months_from_birth'] == ds_birth_combined['max_months_from_birth_before']][['HSHD_SEQNO','HOME_ADM']].drop_duplicates()

hshd_birth_adm['HOME_ADM'] = hshd_birth_adm['HOME_ADM'].astype(str)
hshd_birth_adm['HOME_ADM_2'] = hshd_birth_adm['HOME_ADM'].str[:2]
hshd_birth_adm['HOME_ADM_5'] = hshd_birth_adm['HOME_ADM'].str[:5]
hshd_birth_adm.rename(columns={'HOME_ADM':'HOME_ADM_b'},inplace=True)

ds_birth_combined = ds_birth_combined.merge(hshd_birth_adm, on='HSHD_SEQNO', how='inner')
print(ds_birth_combined.shape)

(4896, 210)


In [89]:
ds_birth_indiv = ds_birth_skt.groupby('KEY').agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'})
ds_birth_combined = ds_birth_combined.merge(ds_birth_indiv, on='KEY', how = 'left')

In [135]:
ds_combined = pd.concat([ds_birth_combined,ds_random_combined], axis=0)
ds_combined['birth event'] = (ds_combined['BS_YR_MON'] == ds_combined['BS_YR_MON_birth']).astype('int')

Heterogeneity 나누기

In [136]:
# Household ICM & TOT_ASST

ds_hshd = ds_combined.groupby(['HSHD_SEQNO','BS_YR_MON']).agg({'ICM':'sum', 'TOT_ASST':'sum'}).reset_index()
ds_hshd = ds_hshd.groupby('HSHD_SEQNO').agg({'ICM':'mean', 'TOT_ASST':'mean'}).reset_index()


for col in kcb_cutoffs.columns:
    ds_hshd[col] = kcb_cutoffs.loc[0,col]

income_condition = [
    ds_hshd['ICM'].between(0, ds_hshd['ICM_cutoff1'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff1'], ds_hshd['ICM_cutoff2'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff2'], ds_hshd['ICM_cutoff3'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff3'], ds_hshd['ICM_cutoff4'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff4'], np.inf, inclusive= 'left')
]

asset_condition = [
    ds_hshd['TOT_ASST'].between(0, ds_hshd['TOT_ASST_cutoff1'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff1'], ds_hshd['TOT_ASST_cutoff2'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff2'], ds_hshd['TOT_ASST_cutoff3'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff3'], ds_hshd['TOT_ASST_cutoff4'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff4'], np.inf, inclusive= 'left')
]

ds_hshd['ICM quantile'] = np.select(income_condition, [1,2,3,4,5], default = np.nan)
ds_hshd['TOT_ASST quantile'] = np.select(asset_condition, [1,2,3,4,5], default = np.nan)
ds_hshd = ds_hshd[['HSHD_SEQNO','ICM quantile','TOT_ASST quantile']]
ds_combined = ds_combined.merge(ds_hshd, on='HSHD_SEQNO', how='left')

In [137]:
ds_combined.shape

(5328, 217)

In [138]:
# Individual SNS, Portal Usage

ds_indiv = ds_combined.groupby('KEY').agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'}).reset_index()
for col in skt_cutoffs:
    ds_indiv[col] = skt_cutoffs[col].iloc[0]


sns_condition = [
    ds_indiv['COMM_SNS_DAY'].between(ds_indiv['sns_cutoff0'], ds_indiv['sns_cutoff1'], inclusive='left'),
    ds_indiv['COMM_SNS_DAY'].between(ds_indiv['sns_cutoff1'], ds_indiv['sns_cutoff2'], inclusive='left'),
]

portal_condition = [
    ds_indiv['INFO_PORTAL_DAY'].between(ds_indiv['portal_cutoff0'], ds_indiv['portal_cutoff1'], inclusive='left'),
    ds_indiv['INFO_PORTAL_DAY'].between(ds_indiv['portal_cutoff1'], ds_indiv['portal_cutoff2'], inclusive='left'),
]

ds_indiv['sns_quantile'] = np.select(sns_condition, ['1', '2'], default=pd.NA)
ds_indiv['portal_quantile'] = np.select(portal_condition, ['1', '2'], default=pd.NA)


ds_indiv = ds_indiv[['KEY','sns_quantile','portal_quantile']]
ds_combined = ds_combined.merge(ds_indiv, on='KEY', how='left')

In [146]:
new_var_name ={
    'CD_USE_AMT':'Card Spending AMT',
    'SIN_CD_USE_AMT':'Credit Card Spending AMT',
    'CHK_CD_USE_AMT': 'Debit Card Spending AMT',
    'CD_FUL_USE_AMT': 'Lump-sum Payment AMT',
    'CD_INSTL_USE_AMT':'Installment Payment AMT',
    'CD_CA_USE_AMT':'Cash Advance AMT',
    'CD_ABRD_USE_AMT':'Overseas Card Spending AMT'
}

ds_combined.rename(columns = new_var_name, inplace = True)

In [ ]:
ds_combined.drop(columns=['hshd_has_no_birth'],inplace=True)
ds_combined.to_stata(os.path.join(path1,"intermediary.dta"), write_index=False)

C:\Users\siyoung\AppData\Local\Temp\ipykernel_8972\2627147718.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Card Spending AMT   ->   Card_Spending_AMT
    Credit Card Spending AMT   ->   Credit_Card_Spending_AMT
    Debit Card Spending AMT   ->   Debit_Card_Spending_AMT
    Lump-sum Payment AMT   ->   Lump_sum_Payment_AMT
    Installment Payment AMT   ->   Installment_Payment_AMT
    Cash Advance AMT   ->   Cash_Advance_AMT
    Overseas Card Spending AMT   ->   Overseas_Card_Spending_AMT
    birth event   ->   birth_event
    ICM quantile   ->   ICM_quantile
    TOT_ASST quantile   ->   TOT_ASST_quantile

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  ds_combined.to_stata(os.path.join(path1,"intermediary.dta"), write_index=False)
